In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
import infos
from tkinter import *
from tkinter import messagebox


link_licao = input('Insira o link da aula: ')

tipo_de_licao = input('''Digite 1 se essa lição deve ser inserido no deck de Escrita Japonesa
Digite 2 se essa lição deve ser inserido no deck de Frases em Japonês: ''')

servico = Service()
opcoes = Options()
opcoes.add_argument("--start-maximized")
navegador = webdriver.Chrome(service=servico, options=opcoes)

Insira o link da aula: https://portal.programajaponesonline.com.br/np/t08/expressando-condicao-com-baai/
Digite 1 se essa lição deve ser inserido no deck de Escrita Japonesa
Digite 2 se essa lição deve ser inserido no deck de Frases em Japonês: 3


In [8]:
def esperar_carregar(elemento):
    while len(navegador.find_elements(By.CLASS_NAME, elemento)) < 1:
        time.sleep(1)
    time.sleep(2)

In [9]:
#login no site da PJO
navegador.get(r'https://portal.programajaponesonline.com.br/login/')
esperar_carregar('input')
navegador.find_element(By.ID, 'user_login').send_keys(infos.login_jpo)
navegador.find_element(By.ID, 'user_pass').send_keys(infos.senha_jpo)
navegador.find_element(By.ID, 'wp-submit').click()
esperar_carregar('menu-item')

In [10]:
#entra na lição
navegador.get(link_licao)
esperar_carregar('single-box-content')

lista_botao = navegador.find_elements(By.CLASS_NAME, 'expand-card')
fronts = navegador.find_elements(By.CLASS_NAME, 'card-front')
backs = navegador.find_elements(By.CLASS_NAME, 'card-back')

#expandindo todas as caixas de texto
for botao in lista_botao:
    botao.click() 
    time.sleep(3)
    
#pega as frentes dos cards
lista_frentes = [frente.text for frente in fronts]   

#pega as traseiras dos cards
lista_tras = [tras.text for tras in backs]

In [11]:
#verificando as frentes e as traseiras do cards
print(lista_frentes)
print(lista_tras)

#caso haja algum item vazio, interrompe o código
for item in lista_frentes:
    if item == '':
        navegador.close()

['波[なみ]が 高[たか]い 場[ば] 合[あい]は、 海[うみ]に 入[はい]らないでください。', '雨[あめ]が 降[ふ]った 場[ば] 合[あい]、 運[うん] 動[どう] 会[かい]は 中[ちゅう] 止[し]になります。', '最[さい] 悪[あく]の 場[ば] 合[あい]、 失[しっ] 敗[ぱい]する 可[か] 能[のう] 性[せい]があります。', '洗[せん] 濯[たく] 機[き]が 故[こ] 障[しょう]した 場[ば] 合[あい]、どこに 連[れん] 絡[らく]したらいいんだろうか？', 'テストの 点[てん]が 良[よ]くなかった 場[ば] 合[あい]、 再[さい] 度[ど]テストを 受[う]けなければならない。', '高[こう] 速[そく] 道[どう] 路[ろ]が 混[こ]んでいる 場[ば] 合[あい]は、 一[いっ] 般[ぱん] 道[どう]を 使[つか]った 方[ほう]が 早[はや]く 目[もく] 的[てき] 地[ち]に 着[つ]きますよ。']
['No caso das ondas serem altas, não entre no mar.\n\n波 (なみ) = onda\n高い (たかい) = alta\n場合 (ばあい) = no caso de\n海 (うみ) = mar\n入る (はいる) = entrar', 'No caso de chover, o torneio esportivo será interrompido.\n\n雨が降る (あめがふる) = chover\n場合 (ばあい) = no caso de\n運動会 (うんどうかい) = torneio esportivo\n中止 (ちゅうし) = interrupção\nなります = se tornar', 'No pior dos casos, existe a possibilidade de falhar.\n\n最悪 (さいあく) = pior\n場合 (ばあい) = casos\n失敗する (しっぱいする) = falhar\n可能性があります (かのうせいがあります) = existe a possibilidade de', 'No caso da máquina de lavar roupas quebrar, onde será que é

In [12]:
#verificando visualmente qual a quantidade de frases que serão adicionadas no Anki
print(len(lista_frentes))
print(len(lista_tras))

6
6


In [13]:
#login no ankiweb
navegador.get(r'https://ankiweb.net/account/login')
esperar_carregar('form-control')
campos = navegador.find_elements(By.CLASS_NAME, 'form-control')
campos[0].send_keys(infos.login_anki)
campos[1].send_keys(infos.senha_anki)
navegador.find_element(By.XPATH, '/html/body/div/main/form/div[3]/button').click()
esperar_carregar('btn-outline-secondary')
navegador.get(r'https://ankiuser.net/edit/')
esperar_carregar('form-group')

In [14]:
if tipo_de_licao == '1':
    #opção Escrita Japonesa
    navegador.find_element(By.XPATH, '/html/body/div/main/div[2]/div/div/div[2]/input').send_keys(infos.decks[0], Keys.TAB)

elif tipo_de_licao == '2':
    #opção Frases em Japonês
    navegador.find_element(By.XPATH, '/html/body/div/main/div[2]/div/div/div[2]/input').send_keys(infos.decks[1], Keys.TAB)

else:
    #deck de testes
    navegador.find_element(By.XPATH, '/html/body/div/main/div[2]/div/div/div[2]/input').send_keys(infos.decks[2], Keys.TAB)

time.sleep(3)

In [15]:
#criando o card
for i, botao in enumerate(lista_botao):
    try:
        campos = navegador.find_elements(By.CLASS_NAME, 'form-control')
        campos[0].send_keys(lista_frentes[i])
        campos[1].send_keys(lista_tras[i])
        time.sleep(3)
        navegador.find_element(By.XPATH, '/html/body/div/main/form/button').click()
        time.sleep(3)
    except:
        navegador.find_element(By.XPATH, '/html/body/main/form/button').send_keys(Keys.END)
        time.sleep(3)
        navegador.find_element(By.XPATH, '/html/body/div/main/form/button').click()
        time.sleep(3)

In [16]:
#fechando o navegador
navegador.close()
janela = Tk()
messagebox.showinfo('Programa concluído com sucesso. Os seus cards foram criados!.')
janela.destroy()